In [1]:
# Importar las bibliotecas necesarias
import sys
import os
import pandas as pd
from sklearn.exceptions import NotFittedError
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

#Configuracion de seaborn
sns.set_theme(style='whitegrid', context='paper', palette='muted')

# Agregar el directorio de scripts al path
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'scripts'))

# Importar el logger personalizado
from logger import CustomLogger

# Crear logger
logger = CustomLogger(developer='David')
app_logger = logger.get_logger('app')
errors_logger = logger.get_logger('errors')
visualizations_logger = logger.get_logger('visualizations')
optimization_logger = logger.get_logger('optimization')
results_logger = logger.get_logger('results')




In [2]:
try:
    # Cargar los datos de entrenamiento
    train_data = pd.read_csv('../data/train.csv')
    app_logger.info("Conjunto de datos de entrenamiento cargado exitosamente.")
except FileNotFoundError:
    errors_logger.error("No se pudo encontrar el archivo de datos de entrenamiento.")
    raise
except Exception as e:
    errors_logger.error(f"Error al cargar los datos de entrenamiento: {str(e)}")
    raise

# Cargamos test.csv
try:
    test_data = pd.read_csv('../data/test.csv')
    app_logger.info("Conjunto de datos de test cargado exitosamente.")
except FileNotFoundError:
    errors_logger.error("No se pudo encontrar el archivo de datos de test.")
    raise

# Mostrar las primeras filas del conjunto de datos
train_data.head()


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [3]:
try:
    # Valores faltantes
    print("Valores faltantes por columna:")
    valores_faltantes = train_data.isnull().sum()
    porcentaje_faltantes = 100 * train_data.isnull().sum() / len(train_data)
    tabla_faltantes = pd.concat([valores_faltantes, porcentaje_faltantes], axis=1, keys=['Total', 'Porcentaje'])
    print(tabla_faltantes[tabla_faltantes['Total'] > 0].sort_values('Total', ascending=False))

    # Filas duplicadas
    filas_duplicadas = train_data.duplicated().sum()
    print(f"\nNúmero de filas duplicadas: {filas_duplicadas}")

    # Registrar en el log
    if valores_faltantes.sum() > 0:
        app_logger.info(f"Se encontraron {valores_faltantes.sum()} valores faltantes en total.")
    else:
        app_logger.info("No se encontraron valores faltantes en el conjunto de datos.")

    if filas_duplicadas > 0:
        app_logger.warning(f"Se encontraron {filas_duplicadas} filas duplicadas en el conjunto de datos.")
    else:
        app_logger.info("No se encontraron filas duplicadas en el conjunto de datos.")

except Exception as e:
    errors_logger.exception("Error al analizar valores faltantes y filas duplicadas:")
    app_logger.error("Se produjo un error al analizar los datos. Consulte el registro de errores para más detalles.")

Valores faltantes por columna:
              Total  Porcentaje
PoolQC         1453   99.520548
MiscFeature    1406   96.301370
Alley          1369   93.767123
Fence          1179   80.753425
MasVnrType      872   59.726027
FireplaceQu     690   47.260274
LotFrontage     259   17.739726
GarageType       81    5.547945
GarageYrBlt      81    5.547945
GarageFinish     81    5.547945
GarageQual       81    5.547945
GarageCond       81    5.547945
BsmtFinType2     38    2.602740
BsmtExposure     38    2.602740
BsmtFinType1     37    2.534247
BsmtCond         37    2.534247
BsmtQual         37    2.534247
MasVnrArea        8    0.547945
Electrical        1    0.068493

Número de filas duplicadas: 0


## Missing Columns on Train
- Pool Quality Col has 99,52% missing values, but we still have Pool Area wich i think is more important than PoolQc
- MiscFeature Col has 96.30% missing values. We also have MiscVal that represents the value of Misc Feature.
    - Each Misc Feature has a unique value or has multiple values?? If each MiscFeature has only one unique value (univoque relationship) we can drop MiscFeature and leave Misc Val because we wont lose info, otherwise, drop both. Even if MiscVal has no NaNs, using it on its own may not be the best for the model to gather relationships.
- Alley has 93% missing values, no other col is related to this one so we can directly drop it.


## Verificar si la relación es unívoca entre MiscFeature y MiscVal

In [4]:
try:
    # Crear un DataFrame con MiscFeature y MiscVal
    misc_df = train_data[['MiscFeature', 'MiscVal']]
    app_logger.info("DataFrame misc_df creado exitosamente")
except KeyError as e:
    errors_logger.exception("Error al crear misc_df: Columna no encontrada")
    print(f"Error: No se encontró la columna {str(e)} en el conjunto de datos")
    raise

try:
    # Agrupar por MiscFeature y contar los valores únicos de MiscVal
    relacion_misc = misc_df.groupby('MiscFeature')['MiscVal'].nunique().reset_index()
    relacion_misc.columns = ['MiscFeature', 'Valores_Unicos_MiscVal']
    app_logger.info("Relación entre MiscFeature y MiscVal generada correctamente")
except Exception as e:
    errors_logger.exception("Error al generar la relación entre MiscFeature y MiscVal")
    print("Se produjo un error al analizar la relación. Consulte el registro de errores para más detalles.")
    raise

print("Relación entre MiscFeature y MiscVal:")
print(relacion_misc)

try:
    # Verificar si cada MiscFeature tiene un único valor de MiscVal
    es_univoca = (relacion_misc['Valores_Unicos_MiscVal'] == 1).all()
    
    if es_univoca:
        print("\nLa relación entre MiscFeature y MiscVal es unívoca.")
        app_logger.info("La relación entre MiscFeature y MiscVal es unívoca. Se puede considerar dejar una columna.")
    else:
        print("\nLa relación entre MiscFeature y MiscVal no es unívoca.")
        app_logger.info("La relación entre MiscFeature y MiscVal no es unívoca. Se recomienda eliminar ambas columnas.")
except Exception as e:
    errors_logger.exception("Error al verificar la univocidad de la relación")
    print("Se produjo un error al verificar la relación. Consulte el registro de errores para más detalles.")

if not es_univoca:
    try:
        print("\nEjemplos de MiscFeature con múltiples valores de MiscVal:")
        ejemplos_multiples = relacion_misc[relacion_misc['Valores_Unicos_MiscVal'] > 1]
        for _, row in ejemplos_multiples.iterrows():
            feature = row['MiscFeature']
            valores = misc_df[misc_df['MiscFeature'] == feature]['MiscVal'].unique()
            print(f"MiscFeature: {feature}")
            print(f"Valores de MiscVal: {valores}\n")
            app_logger.info(f"MiscFeature '{feature}' tiene múltiples valores: {valores}")
    except Exception as e:
        errors_logger.exception("Error al mostrar ejemplos de MiscFeature con múltiples valores")
        print("Se produjo un error al mostrar los ejemplos. Consulte el registro de errores para más detalles.")

Relación entre MiscFeature y MiscVal:
  MiscFeature  Valores_Unicos_MiscVal
0        Gar2                       2
1        Othr                       2
2        Shed                      18
3        TenC                       1

La relación entre MiscFeature y MiscVal no es unívoca.

Ejemplos de MiscFeature con múltiples valores de MiscVal:
MiscFeature: Gar2
Valores de MiscVal: [15500  8300]

MiscFeature: Othr
Valores de MiscVal: [3500    0]

MiscFeature: Shed
Valores de MiscVal: [ 700  350  500  400  480  450 1200  800 2000  600 1300   54  620  560
 1400    0 1150 2500]



In [5]:
# Dado que MiscVal y MiscFeature no son univocamente relacionados, se eliminan ambas columnas puesto que estan relacionadas y no se tienen datos de una y la otra son casi todo 0
train_data = train_data.drop(columns=['MiscFeature', 'MiscVal'])
app_logger.info("Se eliminaron las columnas MiscFeature y MiscVal")

In [6]:
try:
    # Filtrar columnas con más del 90% de valores faltantes
    umbral_faltantes = 0.9
    columnas_a_eliminar = tabla_faltantes[tabla_faltantes['Porcentaje'] > 90].index
    train_data_filtrado = train_data.drop(columns=columnas_a_eliminar.drop(['MiscFeature']))

    # Registrar en el log
    if len(columnas_a_eliminar) > 0:
        app_logger.info(f"Se eliminaron {len(columnas_a_eliminar)} columnas con más del 90% de valores faltantes: {', '.join(columnas_a_eliminar)}")
        print(f"Columnas eliminadas: {', '.join(columnas_a_eliminar.drop(['MiscFeature']))}")
    else:
        app_logger.info("No se encontraron columnas con más del 90% de valores faltantes.")
        print("No se encontraron columnas con más del 90% de valores faltantes.")

    # Mostrar la forma del nuevo conjunto de datos
    print(f"\nForma del conjunto de datos original: {train_data.shape}")
    print(f"Forma del conjunto de datos filtrado: {train_data_filtrado.shape}")

except KeyError as e:
    errors_logger.exception(f"Error al acceder a una columna: {str(e)}")
    app_logger.error("Se produjo un error al filtrar las columnas. Verifique los nombres de las columnas.")
except ValueError as e:
    errors_logger.exception(f"Error en el cálculo de porcentajes: {str(e)}")
    app_logger.error("Se produjo un error al calcular los porcentajes de valores faltantes.")
except Exception as e:
    errors_logger.exception(f"Error inesperado: {str(e)}")
    app_logger.error("Se produjo un error inesperado durante el procesamiento de los datos.")


Columnas eliminadas: Alley, PoolQC

Forma del conjunto de datos original: (1460, 79)
Forma del conjunto de datos filtrado: (1460, 77)


In [7]:
# Importar dtale
import dtale

# Crear una instancia de dtale con los datos filtrados
d = dtale.show(train_data_filtrado)

# Mostrar el enlace para acceder a la interfaz de dtale
print("Se ha generado un análisis interactivo con dtale.")
print(f"Por favor, acceda a la siguiente URL para explorar los datos: {d._url}")

# Registrar en el log
app_logger.info("Se ha generado un análisis interactivo utilizando dtale.")


Se ha generado un análisis interactivo con dtale.
Por favor, acceda a la siguiente URL para explorar los datos: http://PortatilDavid:40000


## Analysis D-Tale
- We found important predictive power with the following features ``FullBath_quantile, FullBath, GrLiveArea, GrLiveArea_power, GrLiveArea_quantile,GrLiveArea_robust, GarageCars, OverallQual``
- Maybe add a feature that stratifies if the house has garage or not (Never Mind ``GarageCond`` and ``GarageQual`` already do).
- 81 houses have 0 car capacity in garage and 0 sqft but not NA specified on ``GarageCond``, ``GarageQual``, ``GarageType`` and ``GarageFinish``, drop rows with 0 or impute them. If a house has 0 for ``GarageCars`` and ``GarageArea``, the ``GarageType``, ``GarageCond``, ``GarageQual``, and ``GarageFinish`` should be NA?.
- ``GarageQual`` and ``GarageCond`` are duplicates.
- The features ``Fireplaces`` and ``FireplaceQu`` are both NaN or 0 for the same row, so imputation should be done simultaneosly maintaining their possible relationship, imputing the mean and most frecuent values independently could potentially introduce noise as the pairing could possibly be random.

- Most of the numerical variables where Non-normal, with log relationship with sales price (target).

    ### Low Variance Features
    - Dtale showed the following features having low variance:
        - ``BsmtFinSF2`` : 88.56% are 0s, related to this feature we have ``BsmtFinType2`` with 88.33% of rows with value 'Unf' which means 'unfinished '. This could mean that as it is unfinshed the sqft on ``BsmtFinSF2`` are not taken into account so the value will be 0?. Coincidence percentaje between ``BsmtFinType2 == 'Unf' and BsmtFinSF2 == 0``: 97.14%
        - ``LowQualFinSF``: 98.22% are 0s. This feature represents 'Low quality finished square feet (all floors)', i dont really know if we could impute this values as the meaning of the variable is not deterministic or easy to define because it represent the sqft of LowQuality finished sqft of all floors, but there isnt a feature defining wich houses are LowQuality or finished. 
            - Maybe we could create a feature called ``Total_sqft`` and/or Mean between all floors per house, etc... 
        - ``KitchenAbvGr``: From possible values {0,1,2,3} 95.34% are 1s. These feature represents the number of kitchens above grade. There is also the ``TotRmsAbvGrd`` feature measuring the total rooms above grade and the feature ``KitchenQual`` wich indicates the quality of the kitchens. Due to this i think we could drop this feature and leave the other two, as the model will probably discover the relationships.
        - ``PoolArea``: 99.52% values are 0s. The related feature was ``PoolQC`` representing the pool quality but it had 99.79% of NaNs values so i think we can drop both.
        - ``EnclosedPorch``: 85.75% of 0s.
        - ``3SsnPorch``: 98.36% of 0s.
        - ``ScreenPorch``: 92.05% of 0s.
        - Out of this 3 features realted with sqft of different types of Porch the one that has the most sens on maintaining on the dataset is ``EnclosedPorch`` that is de opposite to ``OpenPorch`` (that measures the open porch sqft) and its not that much full of 0s as the other two. This could be okay with te model as it can learn that if we have some value on ``EnclosedPorch``, the house has an enclosed porch, and the same for ``OpenPorch``. The other 2 features are practically all 0s and not much value can be obtained. I think the 2 important features are the ones i mentioned, we could do some new features to encode this info of having 1 porch or another or both.
        

In [8]:
Low_Variance_Features = [
    "BsmtFinSF2",
    "LowQualFinSF",
    "KitchenAbvGr",
    "PoolArea",
    "EnclosedPorch",
    "3SsnPorch",
    "ScreenPorch"
]

In [9]:
try:
    # Comparar YearRemodAdd con YearBuilt para identificar casas remodeladas
    casas_remodeladas = train_data_filtrado[train_data_filtrado['YearRemodAdd'] != train_data_filtrado['YearBuilt']]
    casas_no_remodeladas = train_data_filtrado[train_data_filtrado['YearRemodAdd'] == train_data_filtrado['YearBuilt']]

    # Calcular el número de casas remodeladas y no remodeladas
    num_remodeladas = len(casas_remodeladas)
    num_no_remodeladas = len(casas_no_remodeladas)

    # Imprimir los resultados
    print(f"Número de casas remodeladas: {num_remodeladas}")
    print(f"Número de casas no remodeladas: {num_no_remodeladas}")
    print(f"Porcentaje de casas remodeladas: {(num_remodeladas / len(train_data_filtrado)) * 100:.2f}%")

    # Registrar en el log
    app_logger.info(f"Se identificaron {num_remodeladas} casas remodeladas y {num_no_remodeladas} casas no remodeladas.")

except KeyError as e:
    errors_logger.exception(f"Error al acceder a las columnas: {str(e)}")
    app_logger.error("No se pudo completar el análisis de casas remodeladas debido a un error en las columnas.")
except ZeroDivisionError:
    errors_logger.exception("Error al calcular el porcentaje: División por cero.")
    app_logger.error("No se pudo calcular el porcentaje de casas remodeladas debido a un error en los datos.")
except Exception as e:
    errors_logger.exception(f"Error inesperado durante el análisis de casas remodeladas: {str(e)}")
    app_logger.error("Se produjo un error inesperado durante el análisis de casas remodeladas.")


Número de casas remodeladas: 696
Número de casas no remodeladas: 764
Porcentaje de casas remodeladas: 47.67%


In [10]:
def calcular_porcentajes(df):
    '''
    Calcula el porcentaje de casas que coinciden en el año de construcción con el año de construcción del garaje y el año de remodelación.
    '''
    try:
        coinciden = df['YearBuilt'] == df['GarageYrBlt']
        diferentes = ~coinciden
        porcentaje_coinciden = (coinciden.sum() / len(df)) * 100
        porcentaje_diferentes = (diferentes.sum() / len(df)) * 100
        return porcentaje_coinciden, porcentaje_diferentes
    except KeyError as e:
        app_logger.error(f"Error al acceder a las columnas: {e}")
        raise
    except Exception as e:
        app_logger.error(f"Error inesperado al calcular porcentajes: {e}")
        raise

In [11]:
def analizar_coincidencias_remod(df, diferentes):
    '''
    Analiza las casas que coinciden en el año de construcción con el año de construcción del garaje y el año de remodelación.
    '''
    try:
        coinciden_con_remod = df[diferentes]['GarageYrBlt'] == df[diferentes]['YearRemodAdd']
        porcentaje_coinciden_remod = (coinciden_con_remod.sum() / diferentes.sum()) * 100
        return porcentaje_coinciden_remod
    except KeyError as e:
        app_logger.error(f"Error al acceder a las columnas para análisis de remodelación: {e}")
        raise
    except Exception as e:
        app_logger.error(f"Error inesperado en análisis de remodelación: {e}")
        raise

In [12]:
def calcular_diferencias_tiempo(df):
    '''
    Calcula las diferencias de tiempo entre el año de construcción del garaje y el año de construcción y el año de remodelación.
    '''
    try:
        df['diff_with_yearbuilt'] = abs(df['GarageYrBlt'] - df['YearBuilt'])
        df['diff_with_yearremodadd'] = abs(df['GarageYrBlt'] - df['YearRemodAdd'])
        return df
    except KeyError as e:
        app_logger.error(f"Error al calcular diferencias de tiempo: {e}")
        raise
    except Exception as e:
        app_logger.error(f"Error inesperado al calcular diferencias de tiempo: {e}")
        raise

In [13]:
def analizar_proximidad(df, umbral=7):
    '''
    Analiza las casas que están dentro del umbral de proximidad con el año de construcción del garaje y el año de construcción y el año de remodelación.
    '''
    try:
        proxima_a_yearbuilt = df[df['diff_with_yearbuilt'] <= umbral]
        proxima_a_yearremodadd = df[df['diff_with_yearremodadd'] <= umbral]
        mas_de_umbral = df[(df['diff_with_yearbuilt'] > umbral) & (df['diff_with_yearremodadd'] > umbral)]
        return proxima_a_yearbuilt, proxima_a_yearremodadd, mas_de_umbral
    except KeyError as e:
        app_logger.error(f"Error al analizar proximidad: {e}")
        raise
    except Exception as e:
        app_logger.error(f"Error inesperado al analizar proximidad: {e}")
        raise

In [14]:
def calcular_tiempos_medios_extremos(df):
    '''
    Calcula los tiempos medios y extremos de la construcción y la remodelación.
    '''
    try:
        tiempo_construccion = df['GarageYrBlt'] - df['YearBuilt']
        tiempo_remodelacion = df['YearRemodAdd'] - df['GarageYrBlt']
        tiempo_hasta_remodelacion = df['YearRemodAdd'] - df['YearBuilt']
        
        return {
            'construccion': (tiempo_construccion.mean(), tiempo_construccion.min(), tiempo_construccion.max()),
            'remodelacion': (tiempo_remodelacion.mean(), tiempo_remodelacion.min(), tiempo_remodelacion.max()),
            'hasta_remodelacion': (tiempo_hasta_remodelacion.mean(), tiempo_hasta_remodelacion.min(), tiempo_hasta_remodelacion.max())
        }
    except KeyError as e:
        app_logger.error(f"Error al calcular tiempos medios y extremos: {e}")
        raise
    except Exception as e:
        app_logger.error(f"Error inesperado al calcular tiempos medios y extremos: {e}")
        raise

In [15]:
def analizar_datos(train_data_filtrado):
    '''
    Analiza los datos de construcción y remodelación.
    '''
    try:
        porcentaje_coinciden, porcentaje_diferentes = calcular_porcentajes(train_data_filtrado)
        diferentes = train_data_filtrado['YearBuilt'] != train_data_filtrado['GarageYrBlt']
        porcentaje_coinciden_remod = analizar_coincidencias_remod(train_data_filtrado, diferentes)
        
        no_coincide_ninguno = (~(train_data_filtrado['YearBuilt'] == train_data_filtrado['GarageYrBlt'])) & \
                              (~(train_data_filtrado['GarageYrBlt'] == train_data_filtrado['YearRemodAdd']))
        porcentaje_no_coincide_ninguno = (no_coincide_ninguno.sum() / len(train_data_filtrado)) * 100
        
        df_no_coincide = calcular_diferencias_tiempo(train_data_filtrado[no_coincide_ninguno])
        proxima_a_yearbuilt, proxima_a_yearremodadd, mas_de_7_anos = analizar_proximidad(df_no_coincide)
        
        porcentaje_proxima_a_yearbuilt = (len(proxima_a_yearbuilt) / len(df_no_coincide)) * 100
        porcentaje_proxima_a_yearremodadd = (len(proxima_a_yearremodadd) / len(df_no_coincide)) * 100
        porcentaje_mas_de_7_anos = (len(mas_de_7_anos) / len(df_no_coincide)) * 100
        porcentaje_mas_7_anos_total = (len(mas_de_7_anos) / len(train_data_filtrado)) * 100
        
        tiempos = calcular_tiempos_medios_extremos(train_data_filtrado)
        
        # Imprimir resultados
        print(f"Porcentaje de casas donde YearBuilt coincide con GarageYrBlt: {porcentaje_coinciden:.2f}%")
        print(f"Del {porcentaje_diferentes:.2f}% que no coincide con YearBuilt, {porcentaje_coinciden_remod:.2f}% coincide con YearRemodAdd")
        print(f"Porcentaje de casas donde GarageYrBlt no coincide ni con YearBuilt ni con YearRemodAdd: {porcentaje_no_coincide_ninguno:.2f}%")
        print(f"Porcentaje de casas donde GarageYrBlt está a más de 7 años de YearBuilt y YearRemodAdd respecto al total: {porcentaje_mas_7_anos_total:.2f}%")
        print(f"Porcentaje de casas donde GarageYrBlt no coincide con YearBuilt ni con YearRemodAdd y está a menos de 7 años de YearBuilt: {porcentaje_proxima_a_yearbuilt:.2f}%")
        print(f"Porcentaje de casas donde GarageYrBlt no coincide con YearBuilt ni con YearRemodAdd y está a menos de 7 años de YearRemodAdd: {porcentaje_proxima_a_yearremodadd:.2f}%")
        print(f"Porcentaje de casas donde GarageYrBlt no coincide con YearBuilt ni con YearRemodAdd y está a más de 7 años de ambas fechas: {porcentaje_mas_de_7_anos:.2f}%")
        
        for key, (media, minimo, maximo) in tiempos.items():
            print(f"Tiempo {key} - Media: {media:.2f}, Mínimo: {minimo:.2f}, Máximo: {maximo:.2f} años")
        
        # Registrar en el log
        app_logger.info(f"Análisis de YearBuilt vs GarageYrBlt completado. Coinciden: {porcentaje_coinciden:.2f}%, Diferentes: {porcentaje_diferentes:.2f}%")
        app_logger.info(f"El {porcentaje_no_coincide_ninguno:.2f}% de las casas tienen GarageYrBlt que no coincide ni con YearBuilt ni con YearRemodAdd.")
        app_logger.info(f"El {porcentaje_mas_7_anos_total:.2f}% del total de casas tienen GarageYrBlt a más de 7 años de YearBuilt y YearRemodAdd.")
        
    except Exception as e:
        app_logger.error(f"Error en el análisis de datos: {e}")
        raise

In [16]:
# Ejecutar el análisis
try:
    analizar_datos(train_data_filtrado)
except Exception as e:
    app_logger.error(f"Error general en la ejecución del análisis: {e}")

Porcentaje de casas donde YearBuilt coincide con GarageYrBlt: 74.59%
Del 25.41% que no coincide con YearBuilt, 14.56% coincide con YearRemodAdd
Porcentaje de casas donde GarageYrBlt no coincide ni con YearBuilt ni con YearRemodAdd: 21.71%
Porcentaje de casas donde GarageYrBlt está a más de 7 años de YearBuilt y YearRemodAdd respecto al total: 9.45%
Porcentaje de casas donde GarageYrBlt no coincide con YearBuilt ni con YearRemodAdd y está a menos de 7 años de YearBuilt: 22.08%
Porcentaje de casas donde GarageYrBlt no coincide con YearBuilt ni con YearRemodAdd y está a menos de 7 años de YearRemodAdd: 23.03%
Porcentaje de casas donde GarageYrBlt no coincide con YearBuilt ni con YearRemodAdd y está a más de 7 años de ambas fechas: 43.53%
Tiempo construccion - Media: 5.55, Mínimo: -10.00, Máximo: 123.00 años
Tiempo remodelacion - Media: 6.93, Mínimo: -53.00, Máximo: 98.00 años
Tiempo hasta_remodelacion - Media: 13.60, Mínimo: 0.00, Máximo: 123.00 años


C:\Users\david\AppData\Local\Temp\ipykernel_11380\2723570695.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\david\AppData\Local\Temp\ipykernel_11380\2723570695.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [17]:
def calcular_porcentaje(numerador, denominador):
    try:
        return (numerador / denominador) * 100
    except ZeroDivisionError:
        app_logger.warning("División por cero al calcular porcentaje")
        return 0

In [18]:
def analizar_bsmt_fin_type2(df):
    try:
        coincidencias = ((df['BsmtFinType2'] == 'Unf') & (df['BsmtFinSF2'] == 0)).sum()
        total_casos_0 = (df['BsmtFinSF2'] == 0).sum()
        porcentaje_coincidencia = calcular_porcentaje(coincidencias, total_casos_0)

        app_logger.info(f"El {porcentaje_coincidencia:.2f}% de los casos tienen BsmtFinType2 == 'Unf' y BsmtFinSF2 == 0 simultáneamente.")
        print(f"Porcentaje de coincidencia entre BsmtFinType2 == 'Unf' y BsmtFinSF2 == 0 del total de casas donde BsmtFinSF2 == 0: {porcentaje_coincidencia:.2f}%")

        no_coincidencias = ((df['BsmtFinType2'] == 'Unf') & (df['BsmtFinSF2'] != 0)).sum()
        total_casos_no_0 = (df['BsmtFinSF2'] != 0).sum()
        porcentaje_no_coincidencia = calcular_porcentaje(no_coincidencias, total_casos_no_0)

        app_logger.info(f"El {porcentaje_no_coincidencia:.2f}% de los casos tienen BsmtFinType2 == 'Unf' pero BsmtFinSF2 != 0. Con un total de {no_coincidencias} casos.")
        print(f"Porcentaje de casos donde BsmtFinType2 == 'Unf' pero BsmtFinSF2 != 0 respecto al total de casas donde BsmtFinSF2 != 0: {porcentaje_no_coincidencia:.2f}%")

        combinaciones_cero = df[df['BsmtFinSF2'] == 0]['BsmtFinType2'].value_counts()
        porcentaje_combinaciones = calcular_porcentaje(combinaciones_cero, len(df))

        print("Combinaciones con BsmtFinSF2 == 0:")
        for tipo, conteo in combinaciones_cero.items():
            porcentaje = porcentaje_combinaciones[tipo]
            print(f"BsmtFinType2 = '{tipo}': {conteo} casos ({porcentaje:.2f}%)")
            app_logger.info(f"BsmtFinType2 = '{tipo}': {porcentaje:.2f}% de los casos")

        porcentaje_total_cero = (df['BsmtFinSF2'] == 0).mean() * 100
        print(f"\nPorcentaje total de casos con BsmtFinSF2 == 0: {porcentaje_total_cero:.2f}%")
        app_logger.info(f"El {porcentaje_total_cero:.2f}% de los casos tienen BsmtFinSF2 == 0")

    except KeyError as e:
        app_logger.error(f"Error al acceder a las columnas del DataFrame: {e}")
    except Exception as e:
        app_logger.error(f"Error inesperado durante el análisis: {e}")

In [19]:
# Ejecutar el análisis
try:
    analizar_bsmt_fin_type2(train_data_filtrado)
except Exception as e:
    app_logger.error(f"Error general en la ejecución del análisis: {e}")

Porcentaje de coincidencia entre BsmtFinType2 == 'Unf' y BsmtFinSF2 == 0 del total de casas donde BsmtFinSF2 == 0: 97.14%
Porcentaje de casos donde BsmtFinType2 == 'Unf' pero BsmtFinSF2 != 0 respecto al total de casas donde BsmtFinSF2 != 0: 0.00%
Combinaciones con BsmtFinSF2 == 0:
BsmtFinType2 = 'Unf': 1256 casos (86.03%)

Porcentaje total de casos con BsmtFinSF2 == 0: 88.56%


## XiCorr Coeficiente
El coeficiente Xicorr (Xi-correlation coefficient) es una medida de asociación entre dos variables que puede ser utilizada como alternativa al coeficiente de correlación de Pearson, especialmente cuando las variables no tienen una relación lineal.

### Definición del Coeficiente Xicorr

El coeficiente Xicorr se basa en la idea de las "permutaciones locales". Se enfoca en la desviación de las permutaciones locales de una relación ideal, y puede ser más robusto frente a distribuciones no normales y relaciones no lineales.

#### 1. Supongamos que tenemos dos variables $X$ e $Y$ con $n$ observaciones cada una.

$$ X = (X_1, X_2, \ldots, X_n) $$
$$ Y = (Y_1, Y_2, \ldots, Y_n) $$


#### 2. Ordenamos ambas variables en orden ascendente.

$$ X_{(1)}, X_{(2)}, \ldots, X_{(n)} $$
$$ Y_{(1)}, Y_{(2)}, \ldots, Y_{(n)} $$

#### 3. Calculamos las posiciones de los valores originales en los vectores ordenados. Denotamos las posiciones de $X_i$ en $X_{(i)}$ como $P_X(i)$ y de $Y_i$ en $Y_{(i)}$ como $P_Y(i)$.

$$ P_X(i) = \text{posición de } X_i \text{ en } X_{(i)} $$
$$ P_Y(i) = \text{posición de } Y_i \text{ en } Y_{(i)} $$

#### 4. Definimos las permutaciones locales $ \pi_X $ y $ \pi_Y $ de $X$ e $Y$ respectivamente, que representan cómo se permutan los valores cuando se ordenan.

#### 5. Calculamos la desviación de estas permutaciones locales de una relación ideal. Esta desviación se mide a través de una función de distancia. Una forma común de definir esta distancia es usar la distancia de Kendall ($ \tau $).

$$ \tau(\pi_X, \pi_Y) = \text{Número de discordancias entre } \pi_X \text{ y } \pi_Y $$

#### 6. Normalizamos esta distancia para obtener el coeficiente Xicorr. La normalización se hace para que el coeficiente esté en el rango [-1, 1], similar a los coeficientes de correlación tradicionales.

$$ \text{Xicorr}(X, Y) = 1 - \frac{2 \tau(\pi_X, \pi_Y)}{n(n-1)/2} $$

Aquí, $ n(n-1)/2 $ es el número total de pares posibles en el conjunto de datos.

### Interpretación del Coeficiente Xicorr

- **Xicorr = 1**: Indica una correlación perfecta y positiva entre las variables $X$ e $Y$.
- **Xicorr = -1**: Indica una correlación perfecta y negativa entre las variables $X$ e $Y$.
- **Xicorr = 0**: Indica que no hay correlación entre las variables $X$ e $Y$.

### Pasos Intuitivos del Cálculo del Coeficiente Xicorr

1. **Ordenación de las Variables**:
   - Imagina que tienes dos listas de números, una para cada variable, $X$ e $Y$. Primero, ordenas cada lista de menor a mayor.
   - Por ejemplo, si $X = [4, 1, 3]$, lo ordenas como $X_{ordenado} = [1, 3, 4]$. Lo mismo haces con $Y$.

2. **Rastreo de las Posiciones Originales**:
   - Luego, haces un seguimiento de las posiciones originales de los elementos en las listas ordenadas. Es decir, determinas dónde estaba originalmente cada valor en la lista desordenada.
   - Por ejemplo, si $X = [4, 1, 3]$ y $X_{ordenado} = [1, 3, 4]$, el valor 1 estaba en la posición 2 originalmente, el 3 en la posición 3 y el 4 en la posición 1.

3. **Comparación de las Permutaciones**:
   - Ahora, haces lo mismo para la variable $Y$. Una vez que tienes las posiciones originales de ambos conjuntos de datos ordenados, puedes compararlas.
   - Imagina que tienes las posiciones originales de $X$ como $\pi_X$ y las posiciones originales de $Y$ como $\pi_Y$.

4. **Medición de la Discordancia**:
   - Comparas estas permutaciones ($\pi_X$ y $\pi_Y$) para ver cuán diferentes son. La discordancia se mide contando cuántas veces un par de elementos está en un orden diferente en $X$ en comparación con $Y$.
   - Por ejemplo, si en $X$ el segundo elemento viene antes que el primero pero en $Y$ el primero viene antes que el segundo, eso es una discordancia.

5. **Calculo de la Distancia de Discordancia**:
   - La distancia de Kendall ($\tau$) es una forma común de medir esta discordancia, contando cuántas discordancias hay entre las posiciones de $X$ e $Y$.
   - Si hay muchas discordancias, significa que las listas están muy desalineadas.

6. **Normalización de la Distancia**:
   - Para obtener el coeficiente Xicorr, normalizas la cantidad de discordancias para que el valor resultante esté en un rango estándar (generalmente de -1 a 1).
   - La fórmula de normalización convierte la cantidad de discordancias en un valor que puede interpretarse como un coeficiente de correlación.

## Feature Selection Statiscal Methods 

In [20]:
import pandas as pd
import numpy as np
from minepy import MINE
import dcor
from scipy.stats import gamma
from scipy import stats
import xicorpy
from sklearn.feature_selection import mutual_info_regression

def calcular_xi_correlacion(x, y):
    resultado = xicorpy.compute_xi_correlation(x, y, get_modified_xi=False, m_nearest_neighbours=3, get_p_values=False)
    return resultado.iloc[0] if isinstance(resultado, (pd.Series, pd.DataFrame)) else resultado

def calcular_correlacion(x, y):
    # Comprobamos si las variables son numéricas
    if not (np.issubdtype(x.dtype, np.number) and np.issubdtype(y.dtype, np.number)):
        raise ValueError("Ambas variables deben ser numéricas")
    
    # Comprobamos la normalidad de las variables
    _, p_value_x = stats.normaltest(x)
    _, p_value_y = stats.normaltest(y)
    
    # Si ambas variables son normales, usamos Pearson
    if p_value_x > 0.05 and p_value_y > 0.05:
        return stats.pearsonr(x, y)[0]
    
    # Comprobamos la monotonicidad de la relación
    is_monotonic = np.all(np.diff(y[np.argsort(x)]) >= 0) or np.all(np.diff(y[np.argsort(x)]) <= 0)
    
    # Si la relación es monótona, usamos Spearman, si no, Kendall
    if is_monotonic:
        return stats.spearmanr(x, y)[0]
    else:
        return stats.kendalltau(x, y)[0]

def calcular_matriz_xi(df):
    columnas = df.columns
    n = len(columnas)
    matriz_xi = np.eye(n)
    for i in range(n):
        for j in range(i+1, n):
            xi_valor = calcular_xi_correlacion(df.iloc[:, i], df.iloc[:, j])
            matriz_xi[i, j] = matriz_xi[j, i] = xi_valor
    return pd.DataFrame(matriz_xi, index=columnas, columns=columnas)

def hsic_gam(X, Y, alph=0.05):
    n = X.shape[0]
    H = np.eye(n) - np.ones((n, n)) / n
    Kc = H @ X @ X.T @ H
    Lc = H @ Y @ Y.T @ H
    testStat = np.trace(Kc @ Lc)
    varHSIC = (Kc * Lc).sum() / (n * (n - 1))
    K = varHSIC * (n - 1) * (n - 2) * (n - 3)
    c = np.sqrt(K) / n
    v = varHSIC * (2 * (n - 2)) / (n * (n - 1))
    alph = 1 - c / (n * (n - 1))
    thresh = gamma.ppf(1 - alph, v / 2, scale=2 / v)
    return testStat, thresh

def calcular_mic(x, y):
    mine = MINE()
    mine.compute_score(x, y)
    return mine.mic()

def analizar_correlaciones(train_data_filtrado):
    app_logger.info("Calculando la matriz de correlación Xi...")
    matriz_xi = calcular_matriz_xi(train_data_filtrado.select_dtypes(include=[np.number]))
    correlaciones_saleprice = matriz_xi['SalePrice'].sort_values(ascending=False).drop('SalePrice')
    top_correlaciones = correlaciones_saleprice.head(10)
    print("Top 10 correlaciones más fuertes con SalePrice:")
    print(top_correlaciones)
    app_logger.info("Se han identificado las 10 correlaciones más fuertes con SalePrice utilizando el método Xi.")

def realizar_prueba_hsic(train_data_filtrado):
    app_logger.info("Realizando prueba de la función HSIC...")
    caracteristicas_numericas = train_data_filtrado.select_dtypes(include=[np.number]).drop(columns=['SalePrice'])
    Y = train_data_filtrado['SalePrice'].values.reshape(-1, 1)
    resultados = {}
    for caracteristica in caracteristicas_numericas:
        X = train_data_filtrado[caracteristica].values.reshape(-1, 1)
        estadistico_prueba, umbral = hsic_gam(X, Y)
        resultados[caracteristica] = {
            'estadistico': estadistico_prueba,
            'umbral': umbral,
            'dependiente': estadistico_prueba > umbral
        }
        if resultados[caracteristica]['dependiente']:
            app_logger.info(f"La variable {caracteristica} y el precio de venta son dependientes según la prueba HSIC.")
        else:
            app_logger.info(f"No se puede rechazar la hipótesis de independencia entre {caracteristica} y el precio de venta.")
    
    caracteristicas_dependientes = {k: v for k, v in resultados.items() if v['dependiente']}
    app_logger.info(f"Se encontraron {len(caracteristicas_dependientes)} características dependientes del precio de venta según la prueba HSIC.")
    return resultados, caracteristicas_dependientes

def analizar_mic_y_correlacion_distancia(train_data_filtrado):
    app_logger.info("Calculando los puntajes MIC y correlación de distancia...")
    columnas_numericas = train_data_filtrado.select_dtypes(include=[np.number]).drop(columns=['SalePrice']).columns
    puntajes_mic = {columna: calcular_mic(train_data_filtrado[columna], train_data_filtrado['SalePrice']) for columna in columnas_numericas}
    puntajes_corr_dist = {columna: dcor.distance_correlation(train_data_filtrado[columna], train_data_filtrado['SalePrice']) for columna in columnas_numericas}
    
    puntajes_mic = sorted(puntajes_mic.items(), key=lambda item: item[1], reverse=True)
    puntajes_corr_dist = sorted(puntajes_corr_dist.items(), key=lambda item: item[1], reverse=True)
    
    app_logger.info("Análisis de MIC y correlación de distancia completado.")
    return puntajes_mic, puntajes_corr_dist

def analizar_correlacion(train_data_filtrado):
    app_logger.info("Analizando la correlación entre las características...")
    
    # Seleccionar solo las columnas numéricas
    datos_numericos = train_data_filtrado.select_dtypes(include=[np.number])
    
    # Calcular las correlaciones con 'SalePrice' usando la función personalizada
    correlaciones_saleprice = {}
    for columna in datos_numericos.columns:
        if columna != 'SalePrice':
            try:
                correlacion = calcular_correlacion(datos_numericos[columna], datos_numericos['SalePrice'])
                correlaciones_saleprice[columna] = correlacion
            except ValueError as e:
                app_logger.warning(f"No se pudo calcular la correlación para {columna}: {str(e)}")
    
    # Ordenar las correlaciones
    correlaciones_saleprice = dict(sorted(correlaciones_saleprice.items(), key=lambda item: abs(item[1]), reverse=True))
    
    # Obtener las 10 características más correlacionadas con 'SalePrice'
    top_10_correlaciones = dict(list(correlaciones_saleprice.items())[:10])
    
    print("Las 10 características más correlacionadas con SalePrice son:")
    for caracteristica, correlacion in top_10_correlaciones.items():
        print(f"{caracteristica}: {correlacion}")
    
    app_logger.info("Análisis de correlación completado.")
    
    return top_10_correlaciones

# Ejecutar la función de análisis de correlación
top_10_correlaciones = analizar_correlacion(train_data_filtrado)

# Ejecutar las funciones
analizar_correlaciones(train_data_filtrado)
resultados_hsic, caracteristicas_dependientes = realizar_prueba_hsic(train_data_filtrado)
puntajes_mic, puntajes_corr_dist = analizar_mic_y_correlacion_distancia(train_data_filtrado)

# Unir los resultados
union_features = set(caracteristicas_dependientes.keys()) | set(dict(puntajes_mic[:10]).keys()) | set(dict(puntajes_corr_dist[:10]).keys())

app_logger.info("Features en la unión de los top 10 de HSIC, MIC y DCOR:")
app_logger.info(union_features)

print("Features en la unión de los top 10 de HSIC, MIC y DCOR:")
print(union_features)

C:\Users\david\AppData\Local\Temp\ipykernel_11380\3479987822.py:20: FutureWarning:

Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead



Top 10 correlaciones más fuertes con SalePrice:
Id              0.228061
PoolArea        0.216643
LowQualFinSF    0.216589
3SsnPorch       0.213059
BsmtFinSF2      0.212266
ScreenPorch     0.207370
BsmtHalfBath    0.204921
KitchenAbvGr    0.193598
LotArea         0.154691
FullBath        0.116227
Name: SalePrice, dtype: float64


c:\Users\david\anaconda3\envs\david_pcc\lib\site-packages\dcor\_fast_dcov_avl.py:554: UserWarning:

Falling back to uncompiled AVL fast distance covariance terms because of TypeError exception raised: No matching definition for argument type(s) array(int64, 1d, C), array(int64, 1d, C), bool. Rembember: only floating point values can be used in the compiled implementations.

c:\Users\david\anaconda3\envs\david_pcc\lib\site-packages\dcor\_dcor_internals.py:188: RuntimeWarning:

overflow encountered in scalar multiply

c:\Users\david\anaconda3\envs\david_pcc\lib\site-packages\dcor\_fast_dcov_avl.py:554: UserWarning:

Falling back to uncompiled AVL fast distance covariance terms because of TypeError exception raised: No matching definition for argument type(s) array(float64, 1d, C), array(int64, 1d, C), bool. Rembember: only floating point values can be used in the compiled implementations.



Features en la unión de los top 10 de HSIC, MIC y DCOR:
{'BsmtHalfBath', 'TotRmsAbvGrd', 'OverallCond', 'KitchenAbvGr', 'GarageYrBlt', 'YearBuilt', 'BedroomAbvGr', 'OverallQual', 'GrLivArea', '1stFlrSF', 'GarageCars', 'MoSold', 'YrSold', 'FullBath', 'MasVnrArea', 'TotalBsmtSF', 'YearRemodAdd', 'BsmtFullBath', 'GarageArea', 'Fireplaces', 'LotFrontage', 'HalfBath'}


## Modelling

In [21]:
# Importar las bibliotecas necesarias
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

In [22]:
# Preparar los datos
try:
    # Seleccionar las características de union_features
    X = train_data_filtrado[list(union_features)]
    y = train_data_filtrado['SalePrice']
except Exception as e:
    errors_logger.error(f"Error al seleccionar las características de union_features: {e}")
    raise

app_logger.info("Características seleccionadas correctamente.")


In [25]:
# Identificar características de baja varianza que están en union_features
caracteristicas_baja_varianza = [feature for feature in Low_Variance_Features if feature in union_features]

if len(caracteristicas_baja_varianza) > 0:
    app_logger.info(f"Se encontraron {len(caracteristicas_baja_varianza)} características de baja varianza en union_features:")
    for caracteristica in caracteristicas_baja_varianza:
        app_logger.info(f"  - {caracteristica}")
else:
    app_logger.info("No se encontraron características de baja varianza de Low_variance_features en union_features.")

# Imprimir resultados
print("Características de baja varianza en union_features:")
print(caracteristicas_baja_varianza)

print("Características de baja varianza totales:")
print(Low_Variance_Features)

Características de baja varianza en union_features:
['KitchenAbvGr']


Eliminamos esta feature por que no aporta información relevante ya que el 95,31% estan a 1.

In [26]:
# Eliminar la característica 'KitchenAbvGr' de X
app_logger.info("Eliminando la característica 'KitchenAbvGr' de X...")
try:
    X = X.drop('KitchenAbvGr', axis=1)
    app_logger.info("La característica 'KitchenAbvGr' ha sido eliminada de X.")
except KeyError:
    app_logger.info("La característica 'KitchenAbvGr' no se encuentra en X.")
except Exception as e:
    errors_logger.error(f"Error al eliminar la característica 'KitchenAbvGr' de X: {e}")
    raise

# Imprimir las características restantes
print("Características restantes en X después de eliminar 'KitchenAbvGr':")
print(X.columns.tolist())


Características restantes en X después de eliminar 'KitchenAbvGr':
['OverallQual', 'GarageArea', 'TotalBsmtSF', 'LotFrontage', 'BedroomAbvGr', 'YearBuilt', 'YearRemodAdd', 'OverallCond', 'MasVnrArea', 'BsmtHalfBath', 'TotRmsAbvGrd', 'GarageCars', 'HalfBath', 'GarageYrBlt', '1stFlrSF', 'Fireplaces', 'YrSold', 'FullBath', 'MoSold', 'BsmtFullBath', 'GrLivArea']


In [27]:
# Análisis de valores faltantes
app_logger.info("Iniciando análisis de valores faltantes...")

try:
    # Calcular el número de valores faltantes por columna
    missing_values = X.isnull().sum()
    
    # Calcular el porcentaje de valores faltantes
    missing_percentage = 100 * missing_values / len(X)
    
    # Crear un DataFrame con los resultados
    missing_table = pd.concat([missing_values, missing_percentage], axis=1, keys=['Valores Faltantes', 'Porcentaje'])
    
    # Ordenar el DataFrame por el número de valores faltantes en orden descendente
    missing_table = missing_table[missing_table['Valores Faltantes'] > 0].sort_values('Valores Faltantes', ascending=False)
    
    if not missing_table.empty:
        app_logger.info("Se encontraron las siguientes variables con valores faltantes:")
        for index, row in missing_table.iterrows():
            app_logger.info(f"  - {index}: {row['Valores Faltantes']} valores faltantes ({row['Porcentaje']:.2f}%)")
        
        print("Variables con valores faltantes:")
        print(missing_table)
    else:
        app_logger.info("No se encontraron variables con valores faltantes.")
        print("No hay variables con valores faltantes.")

except Exception as e:
    errors_logger.error(f"Error durante el análisis de valores faltantes: {e}")
    raise

app_logger.info("Análisis de valores faltantes completado.")


Variables con valores faltantes:
             Valores Faltantes  Porcentaje
LotFrontage                259   17.739726
GarageYrBlt                 81    5.547945
MasVnrArea                   8    0.547945


Tratamiento de nulos de ``GarageYrBlt``

In [28]:
def preprocesar_GarageYrBlt(X):
    try:
        # Rellenar valores nulos de GarageYrBlt con YearBuilt
        X.loc[:, 'GarageYrBlt'] = X.loc[:, 'GarageYrBlt'].fillna(X['YearBuilt'])
        
        # Usar .loc para ajustar GarageYrBlt
        X.loc[:, 'GarageYrBlt'] = X.apply(lambda row: max(row['GarageYrBlt'], row['YearBuilt']), axis=1)
        
        app_logger.info("Se ha utilizado .loc para evitar SettingWithCopyWarning en el preprocesamiento de GarageYrBlt")
        return X
    except KeyError as e:
        app_logger.error(f"Error: Columna no encontrada - {e}")
        raise
    except Exception as e:
        app_logger.error(f"Error inesperado durante el preprocesamiento de GarageYrBlt: {e}")
        raise

In [29]:
try:
    Nans_previous = X['GarageYrBlt'].isna().sum()
    X = preprocesar_GarageYrBlt(X)
    Nans_after = X['GarageYrBlt'].isna().sum()
    app_logger.info(f"Preprocesamiento de GarageYrBlt completado. Nans anteriores: {Nans_previous}, Nans después: {Nans_after}")
except Exception as e:
    app_logger.error(f"Error durante el preprocesamiento de GarageYrBlt: {e}")

Tratamiento de NaNs de ``MasVnrArea``

In [30]:
def determinar_tipo_y_tratar_nans(X, columna):
    try:
        tipo = X[columna].dtype
        
        if np.issubdtype(tipo, np.number):
            if X[columna].nunique() < 20:
                app_logger.info(f"{columna} se considerará categórica debido a que tiene menos de 20 valores únicos.")
                es_categorica = True
            else:
                app_logger.info(f"{columna} se considerará numérica.")
                es_categorica = False
        else:
            app_logger.info(f"{columna} es categórica por su tipo de dato.")
            es_categorica = True
        
        if es_categorica:
            moda = X[columna].mode()[0]
            X[columna].fillna(moda, inplace=True)
            app_logger.info(f"Se han rellenado los NaNs de {columna} con la moda: {moda}")
        else:
            mediana = X[columna].median()
            X[columna].fillna(mediana, inplace=True)
            app_logger.info(f"Se han rellenado los NaNs de {columna} con la mediana: {mediana}")
        
        nans_restantes = X[columna].isna().sum()
        app_logger.info(f"NaNs restantes en {columna} después del tratamiento: {nans_restantes}")
        
    except KeyError:
        app_logger.error(f"La columna {columna} no existe en el DataFrame.")
    except Exception as e:
        app_logger.error(f"Error inesperado al procesar la columna {columna}: {str(e)}")

In [31]:
# Uso de la función
try:
    determinar_tipo_y_tratar_nans(X, 'MasVnrArea')
except Exception as e:
    app_logger.error(f"Error general en el procesamiento: {str(e)}")

Tratamiento de NaNs de ``LotFrontage``

In [32]:
def tratar_nans_lotfrontage(X):
    try:
        mediana_LotFrontage = X['LotFrontage'].median()
        X['LotFrontage'].fillna(mediana_LotFrontage, inplace=True)
        app_logger.info(f"Se han rellenado los NaNs de LotFrontage con la mediana: {mediana_LotFrontage}")
        
        nans_restantes_LotFrontage = X['LotFrontage'].isna().sum()
        app_logger.info(f"NaNs restantes en LotFrontage después del tratamiento: {nans_restantes_LotFrontage}")
        
        return X
    except KeyError:
        app_logger.error("La columna 'LotFrontage' no existe en el DataFrame.")
        raise
    except Exception as e:
        app_logger.error(f"Error inesperado al tratar los NaNs de LotFrontage: {str(e)}")
        raise

In [33]:
# Uso de la función
try:
    X = tratar_nans_lotfrontage(X)
except Exception as e:
    app_logger.error(f"No se pudo completar el tratamiento de NaNs en LotFrontage: {str(e)}")


In [34]:
def procesar_variables_categoricas(X, variables):
    """Convierte las variables especificadas a tipo categórico."""
    try:
        for var in variables:
            X[var] = X[var].astype('category')
            app_logger.info(f"Se ha convertido {var} a tipo categórico")
        return X
    except KeyError as e:
        app_logger.error(f"Error al procesar variable categórica: {e}")
        raise
    except Exception as e:
        app_logger.error(f"Error inesperado al procesar variables categóricas: {e}")
        raise

def crear_nuevas_features(X):
    """Crea nuevas features basadas en años."""
    try:
        X['BuiltUntilSold'] = X['YrSold'] - X['YearBuilt']
        X['BuiltUntilRemod'] = X['YearRemodAdd'] - X['YearBuilt']
        X['HasBeenRemodeled'] = (X['YearRemodAdd'] != X['YearBuilt']).astype(bool)
        app_logger.info("Se han creado las nuevas features: BuiltUntilSold, BuiltUntilRemod y HasBeenRemodeled")
        return X
    except KeyError as e:
        app_logger.error(f"Error al crear nuevas features: Columna no encontrada - {e}")
        raise
    except Exception as e:
        app_logger.error(f"Error inesperado al crear nuevas features: {e}")
        raise

def clasificar_variables(X):
    """Clasifica las variables en numéricas y categóricas."""
    variables_numericas = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
    variables_categoricas = X.select_dtypes(include=['object', 'category']).columns.tolist()
    
    variables_a_excluir = ['YearRemodAdd', 'YearBuilt', 'YrSold'] + variables_cardinales
    variables_numericas = [var for var in variables_numericas if var not in variables_a_excluir]
    
    app_logger.info(f"Variables numéricas: {variables_numericas}")
    app_logger.info(f"Variables categóricas: {variables_categoricas}")
    return variables_numericas, variables_categoricas

def imprimir_estadisticas(X):
    """Imprime estadísticas de las nuevas features."""
    app_logger.info(f"Estadísticas de BuiltUntilSold: \n{X['BuiltUntilSold'].describe()}")
    app_logger.info(f"Estadísticas de BuiltUntilRemod: \n{X['BuiltUntilRemod'].describe()}")
    app_logger.info(f"Proporción de casas remodeladas: {X['HasBeenRemodeled'].mean():.2f}")

In [35]:
# Variables globales
variables_cardinales = ['HalfBath', 'Fireplaces', 'FullBath', 'BedroomAbvGr', 'TotRmsAbvGrd', 'GarageCars', 'BsmtHalfBath', 'BsmtFullBath']
variables_ordinales = ['OverallQual', 'OverallCond']

In [36]:
# Procesamiento principal
try:
    # Asumimos que X es un DataFrame de pandas ya cargado
    X = procesar_variables_categoricas(X, variables_cardinales + variables_ordinales)
    X = crear_nuevas_features(X)
    variables_numericas, variables_categoricas = clasificar_variables(X)
    imprimir_estadisticas(X)
except Exception as e:
    errors_logger.error(f"Error en el procesamiento principal: {e}")

In [37]:
# Importar las bibliotecas necesarias
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import pandas as pd

transformed_data = X.copy()

# Función para aplicar Label Encoding y convertir a tipo category
def aplicar_label_encoding(X, columna):
    try:
        le = LabelEncoder()
        X[columna] = le.fit_transform(X[columna])
        X[columna] = X[columna].astype('category')
        app_logger.info(f"Se aplicó Label Encoding a la columna {columna} y se convirtió a tipo category")
    except Exception as e:
        errors_logger.error(f"Error al aplicar Label Encoding a la columna {columna}: {str(e)}")
    return X

# Función para aplicar One-Hot Encoding
def aplicar_one_hot_encoding(X, columna):
    try:
        ohe = OneHotEncoder(drop='first', sparse_output=False)
        encoded = ohe.fit_transform(X[[columna]])
        encoded_df = pd.DataFrame(encoded, columns=[f"{columna}_{cat}" for cat in ohe.categories_[0][1:]], dtype=bool)
        X = pd.concat([X.drop(columna, axis=1), encoded_df], axis=1)
        app_logger.info(f"Se aplicó One-Hot Encoding a la columna {columna}")
    except Exception as e:
        errors_logger.error(f"Error al aplicar One-Hot Encoding a la columna {columna}: {str(e)}")
    return X

# Iterar sobre las columnas categóricas
for columna in transformed_data.select_dtypes(include=['object', 'category']).columns:
    num_categorias = transformed_data[columna].nunique()
    
    if num_categorias > 6:
        transformed_data = aplicar_label_encoding(transformed_data, columna)
    else:
        transformed_data = aplicar_one_hot_encoding(transformed_data, columna)

app_logger.info("Se ha completado el procesamiento de variables categóricas")

In [38]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, RobustScaler, PowerTransformer, QuantileTransformer
from scipy.stats import shapiro

def select_transformation(data: pd.DataFrame) -> dict:
    """
    Selecciona la transformación apropiada para cada columna numérica del DataFrame.
    
    Args:
        data (pd.DataFrame): DataFrame de entrada.
    
    Returns:
        dict: Diccionario con las transformaciones seleccionadas para cada columna.
    """
    transformations = {}
    numeric_columns = data.select_dtypes(include=['int64', 'float64']).columns
    
    for column in numeric_columns:
        try:
            column_data = data[column].dropna()
            
            if len(column_data) < 3:
                transformations[column] = None
                app_logger.info(f"La columna {column} tiene menos de 3 valores no nulos. No se aplicará transformación.")
                continue
            
            _, p_value = shapiro(column_data)
            skewness = column_data.skew()
            unique_count = column_data.nunique()
            
            if p_value > 0.05:
                transformations[column] = StandardScaler()
                app_logger.info(f"Se seleccionó StandardScaler para la columna {column}")
            elif abs(skewness) > 1:
                transformations[column] = PowerTransformer(method='yeo-johnson')
                app_logger.info(f"Se seleccionó PowerTransformer para la columna {column}")
            elif unique_count < 10:
                transformations[column] = None
                app_logger.info(f"La columna {column} tiene menos de 10 valores únicos. No se aplicará transformación.")
            elif abs(skewness) > 0.5:
                transformations[column] = RobustScaler()
                app_logger.info(f"Se seleccionó RobustScaler para la columna {column}")
            else:
                transformations[column] = QuantileTransformer(output_distribution='normal')
                app_logger.info(f"Se seleccionó QuantileTransformer para la columna {column}")
        except Exception as e:
            errors_logger.error(f"Error al procesar la columna {column}: {str(e)}")
            transformations[column] = None
    
    return transformations

def apply_transformations(data: pd.DataFrame, transformations: dict) -> pd.DataFrame:
    """
    Aplica las transformaciones seleccionadas a las columnas del DataFrame.
    
    Args:
        data (pd.DataFrame): DataFrame de entrada.
        transformations (dict): Diccionario con las transformaciones a aplicar.
    
    Returns:
        pd.DataFrame: DataFrame con las transformaciones aplicadas.
    """
    transformed_data = data.copy()
    
    for column, transformer in transformations.items():
        try:
            if transformer is not None:
                transformed_data[column] = transformer.fit_transform(data[[column]])
                app_logger.info(f"Se aplicó la transformación {type(transformer).__name__} a la columna {column}")
            else:
                app_logger.info(f"No se aplicó transformación a la columna {column}")
        except Exception as e:
            errors_logger.error(f"Error al aplicar la transformación a la columna {column}: {str(e)}")
            transformed_data[column] = data[column]
    
    return transformed_data

try:
    transformations = select_transformation(transformed_data)
    transformed_data = apply_transformations(transformed_data, transformations)
    app_logger.info("Se completó la selección y aplicación de transformaciones")
except Exception as e:
    errors_logger.error(f"Error general en el proceso de transformación: {str(e)}")

def select_transformation(data: pd.DataFrame) -> dict:
    """
    Selecciona la transformación apropiada para cada columna numérica del DataFrame.
    
    Args:
        data (pd.DataFrame): DataFrame de entrada.
    
    Returns:
        dict: Diccionario con las transformaciones seleccionadas para cada columna.
    """
    transformations = {}
    numeric_columns = data.select_dtypes(include=['int64', 'float64']).columns
    
    for column in numeric_columns:
        column_data = data[column].dropna()
        
        if len(column_data) < 3:
            transformations[column] = None
            continue
        
        try:
            _, p_value = shapiro(column_data)
            skewness = column_data.skew()
            unique_count = column_data.nunique()
            
            if p_value > 0.05:
                transformations[column] = StandardScaler()
            elif abs(skewness) > 1:
                transformations[column] = PowerTransformer(method='yeo-johnson')
            elif unique_count < 10:
                transformations[column] = None
            elif abs(skewness) > 0.5:
                transformations[column] = RobustScaler()
            else:
                transformations[column] = QuantileTransformer(output_distribution='normal')
        except Exception as e:
            print(f"Error al procesar la columna {column}: {str(e)}")
            transformations[column] = None
    
    return transformations

def apply_transformations(data: pd.DataFrame, transformations: dict) -> pd.DataFrame:
    """
    Aplica las transformaciones seleccionadas a las columnas del DataFrame.
    
    Args:
        data (pd.DataFrame): DataFrame de entrada.
        transformations (dict): Diccionario con las transformaciones a aplicar.
    
    Returns:
        pd.DataFrame: DataFrame con las transformaciones aplicadas.
    """
    transformed_data = data.copy()
    
    for column, transformer in transformations.items():
        if transformer is not None:
            try:
                transformed_column = transformer.fit_transform(data[[column]])
                suffix = '_' + transformer.__class__.__name__.lower().replace('transformer', '')
                new_column_name = f"{column}{suffix}"
                transformed_data[new_column_name] = transformed_column
            except Exception as e:
                print(f"Error al transformar la columna {column}: {str(e)}")
        else:
            transformed_data[column] = data[column]
    
    return transformed_data

def check_normality(data: pd.DataFrame) -> None:
    """
    Comprueba la normalidad de las columnas numéricas después de las transformaciones.
    
    Args:
        data (pd.DataFrame): DataFrame transformado.
    """
    numeric_columns = data.select_dtypes(include=['int64', 'float64']).columns
    
    for column in numeric_columns:
        column_data = data[column].dropna()
        
        if len(column_data) < 3:
            print(f"La columna {column} tiene menos de 3 valores no nulos.")
            continue
        
        try:
            _, p_value = shapiro(column_data)
            status = "sigue" if p_value > 0.05 else "no sigue"
            print(f"La columna {column} {status} una distribución normal (p-valor: {p_value:.4f})")
        except Exception as e:
            print(f"Error al comprobar la normalidad de la columna {column}: {str(e)}")

# Uso del código mejorado
try:
    transformations = select_transformation(transformed_data)
    transformed_data = apply_transformations(transformed_data, transformations)
    check_normality(transformed_data)
except Exception as e:
    print(f"Error general: {str(e)}")

La columna GarageArea no sigue una distribución normal (p-valor: 0.0000)
La columna TotalBsmtSF no sigue una distribución normal (p-valor: 0.0000)
La columna LotFrontage no sigue una distribución normal (p-valor: 0.0000)
La columna YearBuilt no sigue una distribución normal (p-valor: 0.0000)
La columna YearRemodAdd no sigue una distribución normal (p-valor: 0.0000)
La columna MasVnrArea no sigue una distribución normal (p-valor: 0.0000)
La columna GarageYrBlt no sigue una distribución normal (p-valor: 0.0000)
La columna 1stFlrSF no sigue una distribución normal (p-valor: 0.0009)
La columna YrSold no sigue una distribución normal (p-valor: 0.0000)
La columna MoSold no sigue una distribución normal (p-valor: 0.0000)
La columna GrLivArea no sigue una distribución normal (p-valor: 0.0009)
La columna BuiltUntilSold no sigue una distribución normal (p-valor: 0.0000)
La columna BuiltUntilRemod no sigue una distribución normal (p-valor: 0.0000)
La columna GarageArea_power no sigue una distribu

In [39]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import cross_validate, train_test_split, cross_val_predict
from sklearn.metrics import make_scorer, mean_squared_error, r2_score, mean_absolute_error
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
import time

def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

scoring = {
    'rmse': make_scorer(rmse, greater_is_better=False),
    'mse': 'neg_mean_squared_error',
    'mae': 'neg_mean_absolute_error',
    'r2': 'r2'
}

def plot_residuals(y_true, y_pred, title, version):
    app_logger.info(f'Creando plot de residuos - {title}')
    print('Creando plot de residuos')
    # Gráfico de residuos vs valores predichos
    residuals = y_true - y_pred
    plt.figure(figsize=(10, 6))
    sns.scatterplot(x=y_pred, y=residuals)
    plt.axhline(y=0, color='r', linestyle='--')
    plt.xlabel('Valores predichos')
    plt.ylabel('Residuos')
    plt.title(f'Gráfico de Residuos - {title}')
    plt.tight_layout()

    # Crear subdirectorio para la versión
    version_dir = f'../plots/v{version}'
    os.makedirs(version_dir, exist_ok=True)
    
    path = f'{version_dir}/residuos_{title.replace(" ", "_")}.png'
    plt.savefig(path)
    plt.close()
    logger.log_visualization(f'Gráfico de Residuos - {title}', path)

    # Distribución de los residuos
    app_logger.info(f'Creando distribución de residuos - {title}')
    print('Creando distribución de residuos')
    plt.figure(figsize=(10, 6))
    sns.histplot(residuals, kde=True)
    plt.xlabel('Residuos')
    plt.ylabel('Frecuencia')
    plt.title(f'Distribución de Residuos - {title}')
    plt.tight_layout()
    path = f'{version_dir}/distribucion_residuos_{title.replace(" ", "_")}.png'
    plt.savefig(path)
    plt.close()
    logger.log_visualization(f'Distribución de Residuos - {title}', path)


def plot_regression(y_true, y_pred, title, version):
    app_logger.info(f'Creando plot de regresión - {title}')
    print('Creando plot de regresión')
    plt.figure(figsize=(10, 6))
    sns.scatterplot(x=y_true, y=y_pred)
    plt.plot([y_true.min(), y_true.max()], [y_true.min(), y_true.max()], 'r--', lw=2)
    plt.xlabel('True values')
    plt.ylabel('Predicted values')
    plt.title(f'Regression Plot - {title}')
    plt.tight_layout()

    # Crear subdirectorio para la versión
    version_dir = f'../plots/v{version}'
    os.makedirs(version_dir, exist_ok=True)
    
    path = f'{version_dir}/regression_{title.replace(" ", "_")}.png'
    plt.savefig(path)
    plt.close()
    logger.log_visualization(f'Regression Plot - {title}', path)

In [47]:
def evaluar_modelo(nombre, modelo, X, y, transformado=False, version="1.0"):
    '''
    """
    Evalúa un modelo de regresión utilizando validación cruzada y métricas de rendimiento.

    Esta función realiza las siguientes tareas:
    1. Ejecuta una validación cruzada de 5 folds en el modelo proporcionado.
    2. Entrena el modelo en todo el conjunto de datos.
    3. Calcula métricas de rendimiento tanto para la validación cruzada como para el conjunto de entrenamiento.
    4. Registra los resultados y genera visualizaciones.

    Parámetros:
    -----------
    ``nombre`` : str
        Nombre del modelo a evaluar.
    ``modelo`` : objeto estimador de scikit-learn
        Modelo de regresión a evaluar.
    ``X`` : array-like o DataFrame
        Características de entrada.
    ``y`` : array-like o Series
        Variable objetivo.
    ``transformado`` : bool, opcional (default=False)
        Indica si los datos han sido transformados previamente.

    Retorna:
    --------
    dict
        Un diccionario con las métricas de rendimiento calculadas.

    Efectos secundarios:
    --------------------
    - Registra los resultados utilizando el logger.
    - Genera y guarda gráficos de residuos y regresión.

    Notas:
    ------
    Las métricas calculadas incluyen RMSE, MSE, MAE y R2, tanto para validación cruzada como para el conjunto de entrenamiento completo.
    """
    '''
    inicio = time.time()
    print(f'Evaluando modelo: {nombre}')
    app_logger.info(f'Evaluando modelo: {nombre} en la version - {version}')
    try:
        
        # Validación cruzada
        resultados_cv = cross_validate(modelo, X, y, cv=5, scoring=scoring, n_jobs=-1, return_train_score=True)
        
        tiempo_ejecucion = time.time() - inicio
        
        # Métricas de validación cruzada
        rmse_cv = -np.mean(resultados_cv['test_rmse'])
        mse_cv = -np.mean(resultados_cv['test_mse'])
        mae_cv = -np.mean(resultados_cv['test_mae'])
        r2_cv = np.mean(resultados_cv['test_r2'])
        
        # Métricas en conjunto de entrenamiento
        rmse_train = -np.mean(resultados_cv['train_rmse'])
        mse_train = -np.mean(resultados_cv['train_mse'])
        mae_train = -np.mean(resultados_cv['train_mae'])
        r2_train = np.mean(resultados_cv['train_r2'])
        
        sufijo = "transformado" if transformado else "sin transformar"
        estudio = f"Regresión simple {nombre} - {sufijo}"
        hiperparams = str(modelo.get_params())
        
        # Registrar resultados
        logger.log_results(estudio, version, nombre, rmse_cv, mse_cv, mae_cv, r2_cv, hiperparams, tiempo_ejecucion)

        # Registrar train
        logger.log_results(estudio + " - Train", version, nombre, rmse_train, mse_train, mae_train, r2_train, hiperparams, tiempo_ejecucion)

        #Realizar inferencia (predicciones) con cross_val_predict
        
        y_pred = cross_val_predict(modelo, X, y, cv=5, n_jobs=-1)
        
        
        # Generar visualizaciones
        plot_residuals(y, y_pred, f'{nombre} - {sufijo}', version)
        plot_regression(y, y_pred, f'{nombre} - {sufijo}', version)
        
        return {
            'nombre': nombre,
            'rmse_cv': rmse_cv,
            'mse_cv': mse_cv,
            'mae_cv': mae_cv,
            'r2_cv': r2_cv,
            'rmse_train': rmse_train,
            'mse_train': mse_train,
            'mae_train': mae_train,
            'r2_train': r2_train,
            'tiempo': tiempo_ejecucion,
            'hiperparams': hiperparams
        }
    
    except Exception as e:
        errors_logger.error(f"Error al evaluar el modelo {nombre}: {str(e)}")
        return None

def evaluar_modelos(X, y, X_transformado, y_transformado, version="1.0"):
    modelos = {
        'LinearRegression': LinearRegression(n_jobs=-1),
        'Ridge': Ridge(random_state=42),
        'Lasso': Lasso(random_state=42),
        'ElasticNet': ElasticNet(random_state=42)
    }
    
    resultados = []
    
    for nombre, modelo in modelos.items():
        resultado = evaluar_modelo(nombre, modelo, X, y, version=version)
        if resultado:
            resultados.append(resultado)
        
        resultado_transformado = evaluar_modelo(nombre, modelo, X_transformado, y_transformado, transformado=True, version=version)
        if resultado_transformado:
            resultados.append(resultado_transformado)
    
    return resultados

In [ ]:
resultados = evaluar_modelos(X, y, transformed_data, y, version="1.0")

In [55]:
transformed_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 47 columns):
 #   Column                        Non-Null Count  Dtype   
---  ------                        --------------  -----   
 0   OverallQual                   1460 non-null   category
 1   GarageArea                    1460 non-null   float64 
 2   TotalBsmtSF                   1460 non-null   float64 
 3   LotFrontage                   1460 non-null   float64 
 4   BedroomAbvGr                  1460 non-null   category
 5   YearBuilt                     1460 non-null   float64 
 6   YearRemodAdd                  1460 non-null   float64 
 7   OverallCond                   1460 non-null   category
 8   MasVnrArea                    1460 non-null   float64 
 9   TotRmsAbvGrd                  1460 non-null   category
 10  GarageYrBlt                   1460 non-null   float64 
 11  1stFlrSF                      1460 non-null   float64 
 12  YrSold                        1460 non-null   in

: 

In [53]:
## Generación de nuevas características y evaluación de modelos
# Función para añadir nuevas características
def añadir_nuevas_caracteristicas(df):
    try:
        nuevas_columnas = {}
        
        for col in df.select_dtypes(include=['int64', 'float64']).columns:
            nuevas_columnas[f'{col}_squared'] = np.square(df[col])
            
            if (df[col] >= 0).all():
                nuevas_columnas[f'{col}_log'] = np.log1p(df[col])
                nuevas_columnas[f'{col}_sqrt'] = np.sqrt(df[col])
            
            # Añadir el producto de columnas
            for col2 in df.select_dtypes(include=['int64', 'float64']).columns:
                if col != col2:
                    nuevas_columnas[f'{col}_x_{col2}'] = df[col] * df[col2]
                    
        df_new = pd.concat([df, pd.DataFrame(nuevas_columnas)], axis=1)
        app_logger.info(f"Se han añadido {len(nuevas_columnas)} nuevas características.")
        return df_new
    except Exception as e:
        app_logger.error(f"Error al añadir nuevas características: {str(e)}")
        raise

try:
    # Aplicar las nuevas características a ambos datasets
    X_new = añadir_nuevas_caracteristicas(X.copy())
    transformed_data_new = añadir_nuevas_caracteristicas(transformed_data.copy())

    # Dividir los nuevos datasets
    X_train_new, X_test_new, y_train_new, y_test_new = train_test_split(X_new, y, test_size=0.2, random_state=42)
    X_train_transformed_new, X_test_transformed_new, y_train_transformed_new, y_test_transformed_new = train_test_split(transformed_data_new, y, test_size=0.2, random_state=42)

    # Evaluar los modelos con los nuevos datasets
    resultados_nuevos = evaluar_modelos(X_train_new, y_train_new, X_train_transformed_new, y_train_transformed_new, version="2.0 - Polynomial Features")

except Exception as e:
    app_logger.error(f"Error en el proceso principal: {str(e)}")



Error in callback <function flush_figures at 0x000001D75B465790> (for post_execute):


KeyboardInterrupt: 

## New Possible Features
- Based on ``YearRemodAdd`` and ``YearBuilt`` as the dataset description stated, if they are equal that means that the house is not remodeled and if its different means that it has been remodeled, we can add a binary feature indicating this.
- Reduce ``YearBuilt`` and ``YrSold`` to ``TimeToSell``.
- Convert ``YearRemodAdd`` to ``TimeUntilRemod`` that means the time since it was built until it was remod, and ``RemodUntilSale`` that is the time since it was remod until it was sold.
- Porch and Deck Areas: Create a total porch area feature and a binary indicator for houses with porches.
- Proximity and Neighborhood Effects: Group neighborhoods into clusters based on median house prices to capture locality effects.
- GeoCode neighborhoods.
- Total Square Footage: Combine all square footage features (``1stFlrSF``, ``2ndFlrSF``, ``TotalBsmtSF``, etc.) into a single feature.

## Inconsistencies
- In some cases ``GarageYrBlt`` (year that the garage was built) was previous to ``YearBuilt`` which is not logical. We can modify this cases and transform those values to the year that the house was built assumming this criterion. This variable is related mostly with the built year and the Remodelation year that we can discard it as it only adds complexity with no info to the dataset.